# Good reference
http://www.diveintopython3.net/special-method-names.html

# Load

1. RDBMS (sqlite, MySQL, PostgreSQL...)
2. NoSQL (MongoDB, Elasticsearch, HBase...)
3. File (csv, json..etc)

In [1]:
import sqlite3

In [2]:
db = sqlite3.connect("./mydb.sqlite3")

In [3]:
db.cursor()

In [4]:
cur = db.cursor()

In [5]:
cur.execute("CREATE TABLE water(wellid int, level int);")

In [6]:
db.commit()

In [23]:
cur.execute("INSERT INTO water VALUES (1, 90)")

In [24]:
db.commit()

In [47]:
cur.execute("SELECT * FROM water;")

In [48]:
for row in cur:
    print(row, row[0], row[1])

(1, 90) 1 90
(1, 90) 1 90
(1, 90) 1 90
(1, 90) 1 90
(1, 90) 1 90
(1, 90) 1 90
(1, 90) 1 90
(1, 90) 1 90
(1, 90) 1 90
(1, 90) 1 90


In [49]:
data_list = [(1, 80),
             (2, 60),
             (3, 20),
             (4, 10),
             (5, 0),
             (6, 80)]

In [50]:
cur.executemany("INSERT INTO water VALUES (?, ?)", data_list)

In [51]:
db.commit()

In [52]:
cur.execute("SELECT * FROM water;")
for row in cur:
    print(row)

(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 80)
(2, 60)
(3, 20)
(4, 10)
(5, 0)
(6, 80)


In [53]:
cur.execute("INSERT INTO water VALUES (%s, %s)" % (1,2))

In [56]:
db.commit()

In [57]:
cur.execute("SELECT * FROM water;")
for row in cur:
    print(row)

(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 90)
(1, 80)
(2, 60)
(3, 20)
(4, 10)
(5, 0)
(6, 80)
(1, 2)


In [58]:
!pip install sqlalchemy

In [61]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

In [105]:
Base = declarative_base()
class User(Base):
    __tablename__ = 'users'
    
    _id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s')" % (self.name, self.fullname)

In [108]:
ian = User(name='david', fullname='david chen')

In [109]:
ian.name

'david'

In [110]:
ian

<User(name='david', fullname='david chen')

In [71]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [73]:
engine = create_engine('sqlite:///mydb.sqlite3', echo=True)

In [74]:
Session = sessionmaker()

In [76]:
Session.configure(bind=engine)

In [77]:
Session

sessionmaker(class_='Session',bind=Engine(sqlite:///mydb.sqlite3), autoflush=True, autocommit=False, expire_on_commit=True)

In [78]:
s = Session()

In [79]:
s

In [80]:
s.add(ian)

In [81]:
# 還沒創建table
s.commit()

2017-05-02 09:58:28,349 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-05-02 09:58:28,350 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 09:58:28,352 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-05-02 09:58:28,353 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 09:58:28,354 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-05-02 09:58:28,356 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2017-05-02 09:58:28,356 INFO sqlalchemy.engine.base.Engine ('ian', 'ian chen')
2017-05-02 09:58:28,358 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) no such table: users [SQL: 'INSERT INTO users (name, fullname) VALUES (?, ?)'] [parameters: ('ian', 'ian chen')]

In [86]:
# 自動建立table
Base.metadata.create_all(engine)

2017-05-02 10:00:07,038 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-05-02 10:00:07,039 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 10:00:07,041 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	_id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (_id)
)


2017-05-02 10:00:07,042 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 10:00:07,043 INFO sqlalchemy.engine.base.Engine COMMIT


In [88]:
s.rollback()

/Users/ian/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sqlalchemy/orm/session.py:507: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "


In [89]:
s.add(ian)
s.commit()

2017-05-02 10:01:19,609 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-05-02 10:01:19,611 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2017-05-02 10:01:19,612 INFO sqlalchemy.engine.base.Engine ('ian', 'ian chen')
2017-05-02 10:01:19,613 INFO sqlalchemy.engine.base.Engine COMMIT


In [91]:
for row in s.query(User).filter_by(name='ian'):
    print(row, row.fullname)

2017-05-02 10:02:47,240 INFO sqlalchemy.engine.base.Engine SELECT users._id AS users__id, users.name AS users_name, users.fullname AS users_fullname 
FROM users 
WHERE users.name = ?
2017-05-02 10:02:47,241 INFO sqlalchemy.engine.base.Engine ('ian',)
<__main__.User object at 0x102d5f438> ian chen


In [97]:
s.add(ian)

In [98]:
s.commit()

2017-05-02 10:07:57,295 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2017-05-02 10:07:57,296 INFO sqlalchemy.engine.base.Engine ('andy', 'andy chen')
2017-05-02 10:07:57,298 INFO sqlalchemy.engine.base.Engine COMMIT


In [111]:
for row in s.query(User).filter_by(name='andy'):
    print(row, row.fullname)

2017-05-02 10:10:39,309 INFO sqlalchemy.engine.base.Engine SELECT users._id AS users__id, users.name AS users_name, users.fullname AS users_fullname 
FROM users 
WHERE users.name = ?
2017-05-02 10:10:39,310 INFO sqlalchemy.engine.base.Engine ('andy',)
<User(name='andy', fullname='andy chen') andy chen


In [100]:
ian

Hi this is a user object

In [101]:
ian

Hi this is a user object

In [102]:
ian

Hi this is a user object

In [103]:
import requests

In [104]:
requests.get("https://google.com")

<Response [200]>

In [ ]:
"<Response [%s]" % (self.status_code)

In [113]:
!pip install pandas
import pandas as pd

In [115]:
df = pd.read_sql("SELECT * FROM users;", create_engine('sqlite:///mydb.sqlite3', echo=True))

2017-05-02 10:17:50,901 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-05-02 10:17:50,902 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 10:17:50,903 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-05-02 10:17:50,904 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 10:17:50,904 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("SELECT * FROM users;")
2017-05-02 10:17:50,905 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 10:17:50,906 INFO sqlalchemy.engine.base.Engine SELECT * FROM users;
2017-05-02 10:17:50,907 INFO sqlalchemy.engine.base.Engine ()


In [116]:
df

,_id,name,fullname
0,1,ian,ian chen
1,2,andy,andy chen


In [134]:
pd.DataFrame([{
    "_id": 3,
    "name": 'brian',
    "fullname": 'brian luan'
}])

,_id,fullname,name
0,3,brian luan,brian


In [135]:
df2 = df.append(pd.DataFrame([{
    "_id": 3,
    "name": 'brian',
    "fullname": 'brian luan'
}]))

In [137]:
df2.index = df2._id

In [138]:
df2

,_id,fullname,name
_id,,,
1,1,ian chen,ian
2,2,andy chen,andy
3,3,brian luan,brian


In [129]:
df2.to_sql('user_clean', create_engine('sqlite:///mydb.sqlite3', echo=True))

2017-05-02 10:22:24,182 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-05-02 10:22:24,183 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 10:22:24,184 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-05-02 10:22:24,185 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 10:22:24,186 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("user_clean")
2017-05-02 10:22:24,187 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 10:22:24,189 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE user_clean (
	"index" BIGINT, 
	_id BIGINT, 
	fullname TEXT, 
	name TEXT
)


2017-05-02 10:22:24,190 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 10:22:24,191 INFO sqlalchemy.engine.base.Engine COMMIT
2017-05-02 10:22:24,192 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_user_clean_index ON user_clean ("index")
2017-05-02 10:22:24,193 INFO sqlalchemy.engine.base.Engine ()
2017-05-02 10:22:24,194 INFO 